I used the given example codes in the lecture such as [Lecture1_Example2](https://github.com/JSFRi/DE1-spark/blob/main/DE1-2023/Lecture1_Example2_RDD_Gutenborg.ipynb). I made the neccesary modification to the tasks and I also used the pyspark documentation as well as stackoverflow for ideas.

In [ ]:
# Starting spark session 
from pyspark.sql import SparkSession
from pyspark.sql import *
spark_session = SparkSession.builder\
    .master("local") \
    .appName("group1-test")\
    .getOrCreate()
## Modification for final deployment

In [ ]:
sc = spark_session.sparkContext
sc.setLogLevel("ERROR")

In [ ]:
#READ DATA- Modification is neccesarry to final deployment
df=spark_session.read.json("RC_2006-04")

In [ ]:
df.printSchema()

In [ ]:
#dropping unnecesary data elements
df.drop('author','author_flair_css_class', 'author_flair_text', 'edited', 'gilded', 'id', 'link_id', 'parent_id', 'retrieved_on', 'stickied', 'subreddit', 'subreddit_id') \
  .printSchema()

In [ ]:
from pyspark.sql import functions as f
from pyspark.sql.functions import col,lit
from datetime import datetime

#Convert epoch to date and group by 
date = df.withColumn("timestamp",f.to_timestamp(df['created_utc']))

In [ ]:
from IPython.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>div.output_scroll { height: 44em; }</style>"))

In [ ]:
#Procedur for periodic distribution and trending over time
print("Give on of the following: \"hour\", \"day\", \"month\", \"year\"")
name=input()
while name not in ["hour", "day", "month", "year"]:
    clear_output(wait=False)
    print("Give on of the following: \"hour\", \"day\", \"month\", \"year\"")
    name=input()

if name=="hour":
    func=f.hour
    form='yyyy-MM-dd HH'
elif name=="day":
    func=f.dayofmonth
    form='yyyy-MM-dd'
elif name=="month":
    func=f.month
    form='yyyy-MM'
elif name=="year":
    func=f.year
    form='yyyy'
    
## distribution of comments inside a day
clear_output(wait=False)
df2=date.groupBy(func("timestamp").alias(name)).count()
d=df2.toPandas()
d.sort_values(by=name)
d.plot(x=name, y='count', style='bo', title="Distribution of comments inside a "+name)

## Change over period (day) overall comments
df2=date.select(f.date_format('timestamp',form).alias(name)).groupby(name).count()
d=df2.toPandas()
d.sort_values(by=name)
d.plot(x=name, y='count', style='--ro', title="Trending per "+name +" (overall worldcount)")

In [ ]:
## Getting words separated, get rid of special characters and convert to lowercaese for analysis

df3=date.withColumn("body", f.regexp_replace(date.body, "[^a-zA-Z0-9]", " "));
raw_words=df3.withColumn('body',f.explode(f.split('body',' ')))
words = raw_words.withColumn('body', f.lower(f.trim(raw_words.body)))


#Read searched word and filter data for it
print("Give the searched key word:")
word=str.lower(input())
dff=words.filter(words.body==word)


#Same procedur for periodic distribution and trending over time
print("Give on of the following: \"hour\", \"day\", \"month\", \"year\"")
name=input()
while name not in ["hour", "day", "month", "year"]:
    clear_output(wait=False)
    print("Give on of the following: \"hour\", \"day\", \"month\", \"year\"")
    name=input()

if name=="hour":
    func=f.hour
if name=="day":
    func=f.dayofmonth
    form='yyyy-MM-dd'
elif name=="month":
    func=f.month
    form='yyyy-MM'
elif name=="year":
    func=f.year
    form='yyyy'
    
## distribution of comments inside a day
clear_output(wait=False)
df2=dff.groupBy(func("timestamp").alias(name)).count()
d=df2.toPandas()
d.sort_values(by=name)
d.plot(x=name, y='count', style='bo', title="Distribution of comments inside a "+name+"\n Wordcount for the word: "+word)

## Change over period (day) overall comments

df2=dff.select(f.date_format('timestamp',form).alias(name)).groupby(name).count()
d=df2.toPandas()
d.sort_values(by=name)
d.plot(x=name, y='count', style='--ro', title="Trending per "+name+"\n Wordcount for the word: "+word)